In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
df_org= pd.read_csv('/content/drive/MyDrive/Finetuning_Dataset.csv')

In [ ]:
df_org = df_org.sample(frac=1.0, random_state=42)

In [ ]:
df_org.head()

,label,content,encoded_labels
288,"HR, Leaves","Dear HR Team, I would like to inquire about th...",8
359,"Development, Review",The team's focus on code quality and maintaina...,2
199,"Sales, Not Interested",Thank you for reaching out regarding [project/...,0
306,"HR, Leaves","Dear HR, I am writing to request a leave of a...",8
3,"HR, Recruitment",I'm a student in seeking internship opportun...,1


In [ ]:
labels = df_org['label'].unique().tolist()
labels = [s.strip() for s in labels ]
labels

['HR, Leaves',
 'Development, Review',
 'Sales, Not Interested',
 'HR, Recruitment',
 'HR,Appraisal',
 'Development,Modification',
 'Sales,Termination',
 'Sales, Interested',
 'Sales,Price negotiation']

In [ ]:
for key, value in enumerate(labels):
    print(value)

Sales, Not Interested
HR, Recruitment
Development, Review
HR,Appraisal
Development,Modification
Sales,Price negotiation
Sales, Interested
Sales,Termination
HR, Leaves


In [ ]:
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [ ]:
df_org=df_org.drop(columns=['Unnamed: 0'])

In [ ]:
label2id

{'Sales, Not Interested': 0,
 'HR, Recruitment': 1,
 'Development, Review': 2,
 'HR,Appraisal': 3,
 'Development,Modification': 4,
 'Sales,Price negotiation': 5,
 'Sales, Interested': 6,
 'Sales,Termination': 7,
 'HR, Leaves': 8}

In [ ]:
id2label

{0: 'Sales, Not Interested',
 1: 'HR, Recruitment',
 2: 'Development, Review',
 3: 'HR,Appraisal',
 4: 'Development,Modification',
 5: 'Sales,Price negotiation',
 6: 'Sales, Interested',
 7: 'Sales,Termination',
 8: 'HR, Leaves'}

In [ ]:
df_org["encoded_labels"]=df_org.label.map(lambda x: label2id[x.strip()])

In [ ]:
df_org.head()

,label,content,encoded_labels
163,"Sales, Not Interested","Dear [Contact Name], Thank you for the additio...",0
33,"HR, Recruitment",I'm a results-oriented developer known for my ...,1
15,"HR, Recruitment",[Client Company Name] has a groundbreaking sof...,1
322,"Development, Review","Dear Development Team, Thank you for your hard...",2
57,"HR,Appraisal","Following my recent performance appraisal, I'd...",3


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
SIZE= df_org.shape[0]

train_texts= list(df_org.content[:SIZE//2])

val_texts=   list(df_org.content[SIZE//2:(3*SIZE)//4 ])

test_texts=  list(df_org.content[(3*SIZE)//4:])

train_labels= list(df_org.encoded_labels[:SIZE//2])

val_labels=   list(df_org.encoded_labels[SIZE//2:(3*SIZE)//4])

test_labels=  list(df_org.encoded_labels[(3*SIZE)//4:])

In [ ]:
len(train_texts)

180

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)

In [ ]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):

    # Extract true labels from the input object
    labels = pred.label_ids

    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)

    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')

    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)

    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='/content/drive/MyDrive/translator',
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    fp16=True,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
pip install transformers[torch]

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [ ]:
pip install accelerate

In [ ]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
     # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics= compute_metrics
)

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.595100,1.171175,0.877778,0.874209,0.879635,0.875032
100,0.615900,0.178786,0.988889,0.988173,0.987654,0.989899


TrainOutput(global_step=120, training_loss=0.9345204909642537, metrics={'train_runtime': 18.8861, 'train_samples_per_second': 95.308, 'train_steps_per_second': 6.354, 'total_flos': 105456605205600.0, 'train_loss': 0.9345204909642537, 'epoch': 10.0})

In [ ]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.036945,1.000000,1.000000,1.000000,1.000000
val,0.084466,0.988889,0.988633,0.990741,0.987654
test,0.136018,0.956044,0.955713,0.962121,0.955651


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

In [ ]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label
    # Since pred_label_idx is a tensor containing a single value (the predicted class index),
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [ ]:
text = "I hope this email finds you well. We are reaching out to request some modifications to our existing project. Specifically, we need adjustments. Please let us know your availability for a meeting to discuss the details and timeline for these modifications. Your prompt response would be greatly appreciated."
# "Machine Learning itself is moving towards more and more automated"
predict(text)

(tensor([[0.8142, 0.0083, 0.0175, 0.0037, 0.0058, 0.0077, 0.0730, 0.0260, 0.0437]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(0, device='cuda:0'),
 'Sales, Not Interested')

In [ ]:
test="Your case studies showcasing the use of AI in personalized learning platforms are incredibly insightful. We're building an adaptive learning app and your expertise in AI-powered education solutions is very attractive. We'd love to discuss potential collaboration."
predict(test)

(tensor([[0.0102, 0.3131, 0.4673, 0.1053, 0.0678, 0.0083, 0.0168, 0.0048, 0.0065]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(2, device='cuda:0'),
 'Development, Review')

In [ ]:
text="We dont like the proposed proposal for the project so we would like to put in on hold"
predict(text)

(tensor([[0.1240, 0.0156, 0.0798, 0.0623, 0.2180, 0.1654, 0.1539, 0.0584, 0.1225]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(4, device='cuda:0'),
 'Development,Modification')

In [ ]:
test="Thank you for your continued efforts to address our questions regarding [Project Name]. We appreciate you taking the time to clarify [Specific question(s)]. However, due to unforeseen circumstances, we need to put this project on hold for now. We apologize for any inconvenience"
predict(test)

(tensor([[0.0960, 0.1589, 0.0716, 0.1219, 0.1296, 0.1222, 0.1158, 0.1512, 0.0329]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(1, device='cuda:0'),
 'HR, Recruitment')

In [ ]:
test="We are not willing to continue your services any further now and end this contract."
predict(test)

(tensor([[0.6104, 0.0295, 0.1155, 0.0124, 0.0174, 0.0562, 0.0197, 0.0295, 0.1094]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(0, device='cuda:0'),
 'Sales, Not Interested')

In [ ]:
test="Thank you for your continued efforts to address our questions regarding [Project Name]. We appreciate you taking the time to clarify [Specific question(s)]. However, due to unforeseen circumstances, we need to put this project on hold for now. We apologize for any inconvenience"
predict(test)

(tensor([[0.0960, 0.1589, 0.0716, 0.1219, 0.1296, 0.1222, 0.1158, 0.1512, 0.0329]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(1, device='cuda:0'),
 'HR, Recruitment')

In [ ]:
test="Looking for a development internship at your firm, so please guide me on how to proceed further and here is my resume for your refrence"
predict(test)

(tensor([[0.0137, 0.8870, 0.0088, 0.0423, 0.0037, 0.0126, 0.0075, 0.0047, 0.0196]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(1, device='cuda:0'),
 'HR, Recruitment')

In [ ]:
test="We would like to engage your services for ongoing maintainenec and regualr updates of our FMCG company website.Please provide a proposal outling the scope of work and associated posts."
predict(test)

(tensor([[0.0609, 0.0242, 0.0138, 0.0315, 0.0539, 0.0176, 0.7757, 0.0140, 0.0083]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(6, device='cuda:0'),
 'Sales, Interested')

In [ ]:
model_path = "Sales-Email-Classification"
x=trainer.save_model(model_path)
y=tokenizer.save_pretrained(model_path)

In [ ]:
!zip -r distilbert.zip "/content/Sales-Email-Classification"


  adding: content/Sales-Email-Classification/ (stored 0%)
  adding: content/Sales-Email-Classification/model.safetensors (deflated 7%)
  adding: content/Sales-Email-Classification/tokenizer.json (deflated 71%)
  adding: content/Sales-Email-Classification/training_args.bin (deflated 51%)
  adding: content/Sales-Email-Classification/config.json (deflated 57%)
  adding: content/Sales-Email-Classification/tokenizer_config.json (deflated 76%)
  adding: content/Sales-Email-Classification/special_tokens_map.json (deflated 42%)
  adding: content/Sales-Email-Classification/vocab.txt (deflated 53%)


In [ ]:
import pickle
with open('model.pkl',"wb") as f:
  pickle.dump(x,f)

In [ ]:
with open('tokenizer.pkl',"wb") as f:
  pickle.dump(y,f)